In [ ]:
import boto3

In [ ]:
cf_client = boto3.client('cloudformation')

In [ ]:
cf_client.list_stacks()

In [ ]:
stack_desc = cf_client.describe_stacks(
    StackName='FifoStack'
)

outputs = stack_desc['Stacks'][0]['Outputs']
print(outputs)

In [ ]:
topic1Arn = [o['OutputValue'] for o in outputs if o['OutputKey'] == 'fifoTopic1Arn'][0] 
print(topic1Arn)
topic2Arn = [o['OutputValue'] for o in outputs if o['OutputKey'] == 'fifoTopic2Arn'][0] 
print(topic2Arn)
topic3Arn = [o['OutputValue'] for o in outputs if o['OutputKey'] == 'topic3Arn'][0] 
print(topic3Arn)
s1QueueUrl = [o['OutputValue'] for o in outputs if o['OutputKey'] == 's1QueueUrl'][0] 
print(s1QueueUrl)
s2QueueUrl = [o['OutputValue'] for o in outputs if o['OutputKey'] == 's2QueueUrl'][0] 
print(s2QueueUrl)

In [ ]:
sns_client = boto3.client('sns')
sqs_client = boto3.client('sqs')

In [ ]:
sns_client.publish(
    TopicArn=topic2Arn,
    Message='{"this":true,"number":1000004}',
    MessageGroupId='x2a',
    MessageAttributes={
        'color': {
            'DataType': 'String',
            'StringValue': 'greenish'
        }
    }
)

In [ ]:
sns_client.publish(
    TopicArn=topic1Arn,
    Message='{"this":false}',
    MessageGroupId='x'
)

In [ ]:
sns_client.publish(
    TopicArn=topic2Arn,
    Message='{"another":true}',
    MessageGroupId='xx'
)

In [ ]:
sqs_response = sqs_client.receive_message(
    QueueUrl=s1QueueUrl
)
print(sqs_response)

if 'Messages'in sqs_response:
    receiptHandle = sqs_response['Messages'][0]['ReceiptHandle']
    print('delete message with handle {}'.format(receiptHandle))
    sqs_client.delete_message(
        QueueUrl=s1QueueUrl,
        ReceiptHandle=sqs_response['Messages'][0]['ReceiptHandle']
    )

In [ ]:
sns_client.publish(
    TopicArn=topic3Arn,
    Message='{"another":true}'
)

In [ ]:
sqs_response = sqs_client.receive_message(
    QueueUrl=s2QueueUrl
)
print(sqs_response)

if 'Messages'in sqs_response:
    receiptHandle = sqs_response['Messages'][0]['ReceiptHandle']
    print('delete message with handle {}'.format(receiptHandle))
    sqs_client.delete_message(
        QueueUrl=s2QueueUrl,
        ReceiptHandle=sqs_response['Messages'][0]['ReceiptHandle']
    )

### Fun with Ordering

In [ ]:
# First enqueue 3 messages in the same message group
colors = ["red","green","blue"]
for color in colors:
    
    sns_client.publish(
        TopicArn=topic2Arn,
        Message='{"this":true,"color":"' + color + '"}',
        MessageGroupId='x',
        MessageAttributes={
            'color': {
                'DataType': 'String',
                'StringValue': color
            }
        }
    )

In [ ]:
# Dequeue and delete the first, dequeue the second without an ack
sqs_response = sqs_client.receive_message(
    QueueUrl=s1QueueUrl
)
print(sqs_response)

if 'Messages'in sqs_response:
    receiptHandle = sqs_response['Messages'][0]['ReceiptHandle']
    print('delete message with handle {}'.format(receiptHandle))
    sqs_client.delete_message(
        QueueUrl=s1QueueUrl,
        ReceiptHandle=sqs_response['Messages'][0]['ReceiptHandle']
    )
    
sqs_response = sqs_client.receive_message(
    QueueUrl=s1QueueUrl
)

print(sqs_response)

In [ ]:
# Dequeue the next - note green is repeatedly received, reappearing after the message visibility timeout expires
# until the DLQ redrive kicks in an moves the message to the DLQ, or the message is deleted
sqs_response = sqs_client.receive_message(
    QueueUrl=s1QueueUrl
)
if 'Messages'in sqs_response:
    print(sqs_response['Messages'][0]['Body'])
else:
    print(sqs_response)

In [ ]:
# This time delete the message
sqs_response = sqs_client.receive_message(
    QueueUrl=s1QueueUrl
)
print(sqs_response)

if 'Messages'in sqs_response:
    print(sqs_response['Messages'][0]['Body'])
    receiptHandle = sqs_response['Messages'][0]['ReceiptHandle']
    print('delete message with handle {}'.format(receiptHandle))
    sqs_client.delete_message(
        QueueUrl=s1QueueUrl,
        ReceiptHandle=sqs_response['Messages'][0]['ReceiptHandle']
    )

In [ ]:
# Now dequeue again - blue is picked up
# This time delete the message
sqs_response = sqs_client.receive_message(
    QueueUrl=s1QueueUrl
)
print(sqs_response)

if 'Messages'in sqs_response:
    print(sqs_response['Messages'][0]['Body'])
    receiptHandle = sqs_response['Messages'][0]['ReceiptHandle']
    print('delete message with handle {}'.format(receiptHandle))
    sqs_client.delete_message(
        QueueUrl=s1QueueUrl,
        ReceiptHandle=sqs_response['Messages'][0]['ReceiptHandle']
    )